In [47]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import gpytorch
import torch
import matplotlib.pyplot as plt

# Prepare Data

In [4]:
data_dir = "../InterpolationBaseline/data/Oct0123_Jan3024"
target_industry_id = "_e49pbOSQseqTE5lu-6NMA"
target_residential_id = "pCPex6DkSdS0f5K2f7jyHg"
surrounding_ids = ["JAJWJnroQCSJz0Dr9uVC1g", "BF_v9KW2Q7ijbrvpKfH6eA", "FIzBmPsyQUKdlgWEPuBtHw", "4RMcyiPkRPma6d38ghlpRA"]

In [12]:
def averaging(df):
    # decompose timestamp
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['year'] = df['timestamp'].dt.year
    df['month'] = df['timestamp'].dt.month
    df['day'] = df['timestamp'].dt.day
    df['weekday'] = df['timestamp'].dt.weekday
    df['hour'] = df['timestamp'].dt.hour

    # average
    df = df.loc[:, ['longitude', 'latitude', 'celsius', 'humidity', 'pressure',
                    'year', 'month', 'day', 'weekday', 'hour', 'pm25']]
    df = df.groupby(['year', 'month', 'day', 'weekday', 'hour']).mean().reset_index(drop=False)

    return df

In [23]:
data_train = []
for id in surrounding_ids:
    df = pd.read_csv(os.path.join(data_dir, id + ".csv"))
    df = averaging(df)
    data_train.append(df.to_numpy())
data_train = np.stack(data_train, axis=0)

df = pd.read_csv(os.path.join(data_dir, target_residential_id + ".csv"))
df = averaging(df)
data_resid = [df.to_numpy()]
data_resid = np.stack(data_resid, axis=0)

df = pd.read_csv(os.path.join(data_dir, target_industry_id + ".csv"))
df = averaging(df)
data_indus = [df.to_numpy()]
data_indus = np.stack(data_indus, axis=0)


print(data_train.shape, data_resid.shape, data_indus.shape)

(4, 2928, 11) (1, 2928, 11) (1, 2928, 11)


In [18]:
df.columns

Index(['year', 'month', 'day', 'weekday', 'hour', 'longitude', 'latitude',
       'celsius', 'humidity', 'pressure', 'pm25'],
      dtype='object')

# Gaussian Process Model

In [31]:
class LocalPeriodicKernel(gpytorch.kernels.Kernel):
    is_stationary = True

    def __init__(self, lp_ard=None, **kwargs):
        super().__init__(**kwargs)
        if lp_ard is not None:
            self.periodickernel = gpytorch.kernels.PeriodicKernel(arg_num_dims=lp_ard)
            self.rbfkernel = gpytorch.kernels.RBFKernel(arg_num_dims=lp_ard)
        else:
            self.periodickernel = gpytorch.kernels.PeriodicKernel()
            self.rbfkernel = gpytorch.kernels.RBFKernel()
        self.localperiodickernel = self.periodickernel * self.rbfkernel

    #kernel function
    def forward(self, x1, x2, **params):
        return self.localperiodickernel(x1, x2, **params)
    
class BaseKernel(gpytorch.kernels.Kernel):
    def __init__(self, matern_ard=None, lp_ard=None, **kwargs):
        super().__init__(**kwargs)
        if matern_ard is not None:
            self.maternkernel = gpytorch.kernels.MaternKernel(nu=0.5,ard_num_dims=matern_ard)
        else:
            self.maternkernel = gpytorch.kernels.MaternKernel(nu=0.5)
        if lp_ard is not None:
            self.localperiodickernel = LocalPeriodicKernel(lp_ard=lp_ard)
        else:
            self.localperiodickernel = LocalPeriodicKernel()

    def forward(self, x1, x2, **params):
        # separate the input into continuous and periodic components
        x1_per = x1[:, :4]
        x1_cont = x1[:, 4:]
        x2_per = x2[:, :4]
        x2_cont = x2[:, 4:]
        return self.maternkernel(x1_cont, x2_cont, **params) * self.localperiodickernel(x1_per, x2_per, **params)

class GlobalKernel(gpytorch.kernels.Kernel):
    is_stationary = True

    def __init__(self, matern_ard=None, lp_ard=None, **kwargs):
        super().__init__(**kwargs)

        # base kernel
        self.basekernel = BaseKernel(matern_ard=matern_ard, lp_ard=lp_ard)

        # scale kernel
        self.scalekernel = gpytorch.kernels.ScaleKernel(self.basekernel)

    
    def forward(self, x1, x2, **params):
        return self.scalekernel(x1, x2, **params)
    
class AirGP(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood, matern_ard=None, lp_ard=None):
        super(AirGP, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = GlobalKernel(matern_ard=matern_ard, lp_ard=lp_ard)

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# Training

In [34]:
X = torch.from_numpy(data_train[:, :, 1:-1])
Y = torch.from_numpy(data_train[:, :, -1])
X_resid = torch.from_numpy(data_resid[:, :, 1:-1])
Y_resid = torch.from_numpy(data_resid[:, :, -1])
X_indus = torch.from_numpy(data_indus[:, :, 1:-1])
Y_indus = torch.from_numpy(data_indus[:, :, -1])

n_stations, n_steps, n_features = X.shape
print(X.shape, Y.shape)

torch.Size([4, 2928, 9]) torch.Size([4, 2928])


In [37]:
Y_resid_pred_all = []
Y_indus_pred_all = []
Y_resid_true_all = []
Y_indus_true_all = []

for t in tqdm(range(n_steps)):
    X_train = X[:, t, :]
    Y_train = Y[:, t]
    X_test_resid = X_resid[:, t, :]
    Y_test_resid = Y_resid[:, t]
    X_test_indus = X_indus[:, t, :]
    Y_test_indus = Y_indus[:, t]

    # prepare training
    likelihood = gpytorch.likelihoods.GaussianLikelihood()
    model = AirGP(X_train, Y_train, likelihood, matern_ard=5, lp_ard=4)
    training_iter = 1000
    optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

    # training
    for iter in range(training_iter):
        model.train()
        likelihood.train()
        optimizer.zero_grad()
        output = model(X_train)
        loss = -mll(output, Y_train)
        loss.backward()
        optimizer.step()

    # evaluation
    model.eval()
    likelihood.eval()
    Y_resid_pred = model(X_test_resid).mean.detach().numpy()
    Y_indus_pred = model(X_test_indus).mean.detach().numpy()
    Y_resid_pred_all.append(Y_resid_pred)
    Y_indus_pred_all.append(Y_indus_pred)
    Y_resid_true_all.append(Y_test_resid.numpy())
    Y_indus_true_all.append(Y_test_indus.numpy())

100%|██████████| 2928/2928 [1:23:16<00:00,  1.71s/it]


In [38]:
def metrics_summary(y_true, y_pred):
    rmse = np.sqrt(np.mean((y_true - y_pred) ** 2))
    cvrmse = rmse / np.mean(y_true)
    mae = np.mean(np.abs(y_true - y_pred))
    r2 = 1 - np.sum((y_true - y_pred) ** 2) / np.sum((y_true - np.mean(y_true)) ** 2)
    return rmse, cvrmse, mae, r2

In [43]:
Y_resid_pred_all = np.concatenate(Y_resid_pred_all, axis=0)
Y_indus_pred_all = np.concatenate(Y_indus_pred_all, axis=0)
Y_resid_true_all = np.concatenate(Y_resid_true_all, axis=0)
Y_indus_true_all = np.concatenate(Y_indus_true_all, axis=0)

rmse_resid, cvrmse_resid, mae_resid, r2_resid = metrics_summary(Y_resid_true_all, Y_resid_pred_all)
rmse_indus, cvrmse_indus, mae_indus, r2_indus = metrics_summary(Y_indus_true_all, Y_indus_pred_all)

print("RMSE:")
print("Residential: {:.2f}  Industry: {:.2f}".format(rmse_resid, rmse_indus))
print("CVRMSE:")
print("Residential: {:.2f}  Industry: {:.2f}".format(cvrmse_resid, cvrmse_indus))
print("MAE:")
print("Residential: {:.2f}  Industry: {:.2f}".format(mae_resid, mae_indus))
print("R2:")
print("Residential: {:.2f}  Industry: {:.2f}".format(r2_resid, r2_indus))

RMSE:
Residential: 7.67  Industry: 4.81
CVRMSE:
Residential: 0.45  Industry: 0.26
MAE:
Residential: 4.20  Industry: 3.49
R2:
Residential: 0.61  Industry: 0.84


# Inverse Distance Weighting

In [45]:
class IDW:
    def __init__(self, X, Y):
        """
        X: (n, d), n is the number of samples, d is the dimension of feature vectors
        Y: (n, ), n is the number of samples
        """
        self.X = X
        self.Y = Y

    def predict(self, X_test, p=2):
        """
        X_test: (m, d), m is the number of test samples, d is the dimension of feature vectors
        p: the power of distance
        """
        # construct distance matrix
        dist_matrix = np.zeros((X_test.shape[0], self.X.shape[0]))
        for i in range(X_test.shape[0]):
            for j in range(self.X.shape[0]):
                dist = np.linalg.norm(X_test[i] - self.X[j])
                dist_matrix[i, j] = dist
        
        # construct weight matrix
        weight_matrix = 1 / np.power(dist_matrix, p)

        # normalize weight matrix
        weight_matrix = weight_matrix / np.sum(weight_matrix, axis=1, keepdims=True)
        self.weight_matrix = weight_matrix

        # predict
        Y_pred = np.matmul(weight_matrix, self.Y)
        return Y_pred


In [46]:
Y_resid_pred_all = []
Y_indus_pred_all = []
Y_resid_true_all = []
Y_indus_true_all = []

for t in tqdm(range(n_steps)):
    X_train = X[:, t, :]
    Y_train = Y[:, t]
    X_test_resid = X_resid[:, t, :]
    Y_test_resid = Y_resid[:, t]
    X_test_indus = X_indus[:, t, :]
    Y_test_indus = Y_indus[:, t]

    idw = IDW(X_train.numpy(), Y_train.numpy())
    Y_resid_pred = idw.predict(X_test_resid.numpy())
    Y_indus_pred = idw.predict(X_test_indus.numpy())
    Y_resid_pred_all.append(Y_resid_pred)
    Y_indus_pred_all.append(Y_indus_pred)
    Y_resid_true_all.append(Y_test_resid.numpy())
    Y_indus_true_all.append(Y_test_indus.numpy())

Y_resid_pred_all = np.concatenate(Y_resid_pred_all, axis=0)
Y_indus_pred_all = np.concatenate(Y_indus_pred_all, axis=0)
Y_resid_true_all = np.concatenate(Y_resid_true_all, axis=0)
Y_indus_true_all = np.concatenate(Y_indus_true_all, axis=0)

rmse_resid, cvrmse_resid, mae_resid, r2_resid = metrics_summary(Y_resid_true_all, Y_resid_pred_all)
rmse_indus, cvrmse_indus, mae_indus, r2_indus = metrics_summary(Y_indus_true_all, Y_indus_pred_all)
print("RMSE:")
print("Residential: {:.2f}  Industry: {:.2f}".format(rmse_resid, rmse_indus))
print("CVRMSE:")
print("Residential: {:.2f}  Industry: {:.2f}".format(cvrmse_resid, cvrmse_indus))
print("MAE:")
print("Residential: {:.2f}  Industry: {:.2f}".format(mae_resid, mae_indus))
print("R2:")
print("Residential: {:.2f}  Industry: {:.2f}".format(r2_resid, r2_indus))

100%|██████████| 2928/2928 [00:00<00:00, 18865.23it/s]

RMSE:
Residential: 8.12  Industry: 6.32
CVRMSE:
Residential: 0.48  Industry: 0.35
MAE:
Residential: 4.94  Industry: 3.91
R2:
Residential: 0.56  Industry: 0.72
